In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import LSTM 
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LeakyReLU

In [20]:
    import tensorflow as tf
    import json
    import xlrd
    import traceback   
    import numpy as np #use to handle numeric data
    import nltk #for nlp purpose
    import pandas as pd #use for file that we read
    import re #to handle regular expression
    from keras.models import load_model #To load model
    from keras import backend as K #to load the backend library that we are using 
    from tensorflow.keras.preprocessing import sequence
    from keras.preprocessing.text import Tokenizer
    from textblob import TextBlob
    from sklearn.model_selection import train_test_split
    from langdetect import detect
    import string 
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import h5py
    from datetime import datetime
    from datetime import timedelta
    import datefinder
    from dateparser.search import search_dates
    import spacy
    from nltk.stem.snowball import SnowballStemmer
    from nltk.tokenize import word_tokenize
    import gensim
    import inflect
    p = inflect.engine()
    from dateutil.parser import parse
        


    nlp = spacy.load('en_core_web_sm')

In [21]:
stopwords = nltk.corpus.stopwords.words('english')

In [22]:
Dataset = pd.read_excel("D:\\bot\\botapi\\botapi\\dataset\\Main.xlsx")

In [23]:
Dataset = Dataset[Dataset['Module']=='leave']

In [24]:
Dataset.head()

,s.no,Data,Module,Quantity,sub_intent_one,sub_intent_two,Unnamed: 6
360,361,Please show my pending leaves,leave,particular,leave_inquiry,specific,leave_update
361,362,Please show my all leaves.,leave,particular,leave_inquiry,specific,leave_taken
362,363,Please tell me the balance of casual leave,leave,particular,leave_inquiry,specific,leave_update
363,364,Please tell me the balance of sick leave,leave,particular,leave_inquiry,specific,leave_update
364,365,Please tell me the balance of annual leave,leave,particular,leave_inquiry,specific,leave_update


In [25]:
Data = Dataset['Data']

# Function to perform pre-processing steps

At first tokenization will be performed then stemming will be performed over tokens

In [26]:
def token_stems(text):      
    tokens=tokenizing(text) 
    stems=stemming(tokens)
    return stems       

# Stemming Function

In [27]:
from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")


def stemming(text):
    
    stems =[stemmer.stem(t) for t in text]
    return stems

# Tokenization Function

In [28]:
def tokenizing(text):
    
    #breaking each word and making them tokens
    tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #storing only alpha tokens
    filtered_tokens=[]
    for token in tokens:
        if (re.search('[a-zA-Z]|\'', token)):
            #if token not in stopwords:
            filtered_tokens.append(token)

    return filtered_tokens

# Making model

In [29]:
tokens = []
for i in Data:
    temp = token_stems(i)
    tokens.append(temp)


In [30]:
#print(docs[1],"\n\n",tokens[1])

In [31]:
Dataset['sub_intent_one'].unique()

array(['leave_inquiry', 'LeaveRequest', 'Leave_Approval',
       'emotion_LeaveRequest'], dtype=object)

In [32]:
x, y = np.asarray(tokens) , np.asarray(Dataset['sub_intent_one'])

In [33]:
le = LabelEncoder()
y = le.fit_transform(y)
y= to_categorical(y)
#y[:10]

In [34]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [35]:
sequences = tok.texts_to_sequences(x)

In [36]:
max_len =200
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

Naive Bayes Algo

xtrain,xtest,ytrain,ytest = train_test_split(sequences_matrix,y,random_state = 100,test_size =0.2)

# Model 

In [37]:
def model():
    
    #Create model by simply calling the Sequential constructor
    
    model = Sequential()
    
    #This layer can only be used as the first layer in a model.
    
    model.add(Embedding(max_words,50,input_length=max_len))
    
    model.add(LSTM(128, input_shape=(x.shape[1:]), activation='tanh', return_sequences=True))
    
    model.add(Dropout(0.2))
    model.add(LSTM(90, input_shape=(x.shape[1:]), activation='tanh', return_sequences=True))
    
    model.add(Dropout(0.2))
    
    #specify number of neurons in lstm layer and the activation function that we want to use.
    model.add(LSTM(70, input_shape=(x.shape[1:]), activation='tanh', return_sequences=True))
    
    model.add(Dropout(0.2))
    
    model.add(LSTM(64, activation='tanh',dropout=0.01))
    
    #specify number of neurons in dense layer and the activation function that we want to use.
    
    model.add(Dense(50, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.2))
    model.add(Dense(35, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.2))
    model.add(Dense(25, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.2))
    
    #and we create our output layer with two nodes as we have 2 class labels.
    
    model.add(Dense(15, activation='relu'))
    
    model.add(Dropout(0.01))
    
    model.add(Dense(10, activation='relu'))
    
    model.add(Dropout(0.01))
    
    model.add(Dense(4, activation='softmax'))
    
    #Now for training, we need to define an optimizer, loss measure and the error metric.
    # We will use the binary_crossentropy as our loss measure. 
    #As for the minimization algorithm, we will use "rmsprop".
    #This optimizer is usually a good choice for recurrent neural networks.
    
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
    return model

In [54]:
m= model()

m.fit(sequences_matrix,y,epochs=20, validation_split=0.1)

Train on 1936 samples, validate on 216 samples
Epoch 1/20
1936/1936 [==============================] - ETA: 4:28 - loss: 1.3861 - acc: 0.593 - ETA: 2:32 - loss: 1.3821 - acc: 0.515 - ETA: 1:50 - loss: 1.3628 - acc: 0.531 - ETA: 1:29 - loss: 1.3383 - acc: 0.531 - ETA: 1:15 - loss: 1.3734 - acc: 0.487 - ETA: 1:07 - loss: 1.3706 - acc: 0.468 - ETA: 1:00 - loss: 1.3690 - acc: 0.446 - ETA: 55s - loss: 1.3550 - acc: 0.460 - ETA: 51s - loss: 1.3488 - acc: 0.45 - ETA: 48s - loss: 1.3485 - acc: 0.45 - ETA: 46s - loss: 1.3342 - acc: 0.47 - ETA: 43s - loss: 1.3277 - acc: 0.46 - ETA: 41s - loss: 1.3176 - acc: 0.46 - ETA: 39s - loss: 1.3043 - acc: 0.47 - ETA: 38s - loss: 1.3089 - acc: 0.46 - ETA: 36s - loss: 1.3044 - acc: 0.45 - ETA: 35s - loss: 1.3041 - acc: 0.45 - ETA: 34s - loss: 1.3055 - acc: 0.45 - ETA: 32s - loss: 1.3028 - acc: 0.45 - ETA: 31s - loss: 1.2994 - acc: 0.44 - ETA: 30s - loss: 1.3001 - acc: 0.44 - ETA: 29s - loss: 1.2941 - acc: 0.45 - ETA: 28s - loss: 1.2946 - acc: 0.45 - ETA: 27s

1936/1936 [==============================] - ETA: 40s - loss: 0.5655 - acc: 0.68 - ETA: 39s - loss: 0.4913 - acc: 0.75 - ETA: 38s - loss: 0.5077 - acc: 0.72 - ETA: 38s - loss: 0.5684 - acc: 0.71 - ETA: 39s - loss: 0.6294 - acc: 0.68 - ETA: 39s - loss: 0.6309 - acc: 0.67 - ETA: 39s - loss: 0.6247 - acc: 0.66 - ETA: 38s - loss: 0.6005 - acc: 0.68 - ETA: 37s - loss: 0.6026 - acc: 0.69 - ETA: 37s - loss: 0.6100 - acc: 0.69 - ETA: 36s - loss: 0.6104 - acc: 0.70 - ETA: 35s - loss: 0.6298 - acc: 0.70 - ETA: 34s - loss: 0.6404 - acc: 0.69 - ETA: 33s - loss: 0.6321 - acc: 0.70 - ETA: 32s - loss: 0.6345 - acc: 0.71 - ETA: 31s - loss: 0.6211 - acc: 0.72 - ETA: 30s - loss: 0.6281 - acc: 0.72 - ETA: 29s - loss: 0.6247 - acc: 0.72 - ETA: 29s - loss: 0.6248 - acc: 0.72 - ETA: 28s - loss: 0.6239 - acc: 0.72 - ETA: 27s - loss: 0.6307 - acc: 0.72 - ETA: 27s - loss: 0.6279 - acc: 0.73 - ETA: 26s - loss: 0.6338 - acc: 0.73 - ETA: 25s - loss: 0.6259 - acc: 0.73 - ETA: 25s - loss: 0.6256 - acc: 0.73 - ETA: 

1936/1936 [==============================] - ETA: 41s - loss: 0.1993 - acc: 0.90 - ETA: 43s - loss: 0.3153 - acc: 0.90 - ETA: 44s - loss: 0.2614 - acc: 0.91 - ETA: 43s - loss: 0.2633 - acc: 0.91 - ETA: 42s - loss: 0.2599 - acc: 0.91 - ETA: 41s - loss: 0.2439 - acc: 0.91 - ETA: 41s - loss: 0.2773 - acc: 0.91 - ETA: 41s - loss: 0.2773 - acc: 0.91 - ETA: 40s - loss: 0.2884 - acc: 0.89 - ETA: 39s - loss: 0.2948 - acc: 0.88 - ETA: 38s - loss: 0.3080 - acc: 0.87 - ETA: 37s - loss: 0.2973 - acc: 0.88 - ETA: 36s - loss: 0.2934 - acc: 0.88 - ETA: 35s - loss: 0.2905 - acc: 0.88 - ETA: 34s - loss: 0.3317 - acc: 0.87 - ETA: 33s - loss: 0.3216 - acc: 0.88 - ETA: 32s - loss: 0.3226 - acc: 0.88 - ETA: 32s - loss: 0.3217 - acc: 0.88 - ETA: 31s - loss: 0.3188 - acc: 0.88 - ETA: 30s - loss: 0.3260 - acc: 0.88 - ETA: 29s - loss: 0.3320 - acc: 0.87 - ETA: 29s - loss: 0.3377 - acc: 0.88 - ETA: 28s - loss: 0.3365 - acc: 0.88 - ETA: 27s - loss: 0.3311 - acc: 0.88 - ETA: 26s - loss: 0.3287 - acc: 0.88 - ETA: 

1936/1936 [==============================] - ETA: 51s - loss: 0.3417 - acc: 0.90 - ETA: 50s - loss: 0.3591 - acc: 0.89 - ETA: 48s - loss: 0.2616 - acc: 0.91 - ETA: 48s - loss: 0.2505 - acc: 0.92 - ETA: 48s - loss: 0.2057 - acc: 0.93 - ETA: 47s - loss: 0.2234 - acc: 0.92 - ETA: 46s - loss: 0.1981 - acc: 0.93 - ETA: 46s - loss: 0.1779 - acc: 0.94 - ETA: 45s - loss: 0.1877 - acc: 0.93 - ETA: 44s - loss: 0.1898 - acc: 0.93 - ETA: 43s - loss: 0.1791 - acc: 0.94 - ETA: 42s - loss: 0.1686 - acc: 0.94 - ETA: 41s - loss: 0.1653 - acc: 0.94 - ETA: 40s - loss: 0.1554 - acc: 0.95 - ETA: 39s - loss: 0.1469 - acc: 0.95 - ETA: 38s - loss: 0.1409 - acc: 0.95 - ETA: 37s - loss: 0.1346 - acc: 0.95 - ETA: 36s - loss: 0.1287 - acc: 0.96 - ETA: 35s - loss: 0.1288 - acc: 0.96 - ETA: 34s - loss: 0.1241 - acc: 0.96 - ETA: 33s - loss: 0.1351 - acc: 0.96 - ETA: 32s - loss: 0.1361 - acc: 0.96 - ETA: 31s - loss: 0.1344 - acc: 0.96 - ETA: 30s - loss: 0.1325 - acc: 0.96 - ETA: 29s - loss: 0.1319 - acc: 0.96 - ETA: 

In [31]:
xtrain,xtest,ytrain,ytest = train_test_split(sequences_matrix,y,random_state = 100,test_size = 0.2)

In [32]:
y_pd = m.predict(xtest)

In [33]:
acc_test = m.evaluate(sequences_matrix,y,verbose=0)

In [34]:
print(acc_test)

[0.13451796508541738, 0.962360594795539]


In [35]:
#save your model
m.save('LeaveModel.h5')

Quer For Leave

In [28]:
from keras.models import load_model
m = load_model('LeaveModel.h5')
max_len = 300

In [29]:
docs= Dataset['Data']

In [30]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)

In [ ]:
x, y = np.asarray(tokens) , np.asarray(Dataset['sub_intent_one'])

In [ ]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [ ]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [ ]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [ ]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
query = "update me about leave approving authority"

In [ ]:
query = "i want leave for today"
query = "i want sick leaves for 2 days"

In [ ]:
query = "Approve all leaves on my behalf"

In [ ]:
query = "how many half days have i availed"
query = "do i have any absent for last month"
query = "how many leaves have i availed"
query = "inform me about how many leaves have i taken"
query = "is my leave approved"
#INQUIRY

In [ ]:
query = 'is office off on 9th november'

In [ ]:
query = 'what is leave encashment'
#query = 'can you please tell me about leave encashment procedure'
#query = 'i want to encash my leaves'

In [ ]:
query = "i am feeling sick"
query = "i dont feel i can work today"
query = "i dont want to come "
query = "will office remain close tomorrow"
query = 'who is my leave approving authority'
#query = 'I won’t be able to come to office today'

In [ ]:
query = 'i want sick leave for today'
query = 'Shiza is not feeling well'

In [ ]:
sent = query

In [ ]:
sen = token_stems(sent)
sen

In [ ]:
sen_test = ([list(sen)])
sen_sequences = tok.texts_to_sequences(sen_test)
sen_sequences_matrix = sequence.pad_sequences(sen_sequences,maxlen=max_len)

In [ ]:
score = m.predict(sen_sequences_matrix)
print(score)
print(max(score[0]))

In [ ]:
print("Inquiry 3")
print("Request 0")
print("Emottional 2")
print("Approval 1")

# Emotional Leave

In [121]:
em_leave = Dataset[Dataset['sub_intent_one']=='emotion_LeaveRequest']

In [122]:
data = em_leave['Data']
x= []

In [123]:
for i in data:
    x.append(token_stems(i))

In [124]:
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(x)

In [125]:
sequences = tok.texts_to_sequences(x)
max_len =200
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [126]:
y = em_leave['sub_intent_two']

In [127]:
le = LabelEncoder()
#y = le.fit_transform(y)

In [128]:
from sklearn import svm

In [136]:
m = svm.SVC(kernel = 'rbf',C=50)

In [137]:
m.fit(sequences_matrix,y)

C:\Users\shiza.abid\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [138]:
xtrain,xtest,ytrain,ytest = train_test_split(sequences_matrix,y,random_state=100,test_size=0.2)

In [139]:
y_pdr = m.predict(xtest)

In [140]:
from sklearn.metrics import accuracy_score

In [141]:
lr_ac=accuracy_score(ytest, y_pdr)

In [142]:
print(lr_ac)

0.981651376146789


In [143]:
test = 'i am not feeling well'
test = 'i am going on picnic tomorrow'
test = 'i had an accident yesterday'
test = 'my grandmother is in hospital'
test = 'i am admitted in hospital'

In [144]:
user_response = test
sen = token_stems(user_response)
#sen_test = ([list(sen)])
print(sen)
sen_sequences = tok.texts_to_sequences([sen])
sen_sequences_matrix = sequence.pad_sequences(sen_sequences,maxlen=max_len)
ans = m.predict(sen_sequences_matrix)
print(ans)

['i', 'am', 'admit', 'in', 'hospit']
['casual']


cross validation code

# Query leave

Leave main model

In [ ]:
leave_intent = leave("i have to go to doctor") 
print(leave_intent)

In [ ]:
def leave(name):
    sub_intent = leave_module(name)
                #for sub_intents
    if(sub_intent == 'EmotionlLeave'):
        sub_intent_two = emotional_leave(name)
                    #return sub_intent_two
    elif(sub_intent == 'LeaveApproval'):
        sub_intent_two = leave_approval(name)
                    #return sub_intent_two
    elif(sub_intent == 'LeaveRequest'):
        sub_intent_two = classify_leaverequest(name)
    else:
        sub_intent_two = classify_leaveinquiry(name)
    return(sub_intent_two)

In [ ]:
def leave_module(name):
        sub_intent = ''
        train = pd.read_excel("D:\\virtual\\leave_final.xlsx")
        docs= train['Data']
        tokens = []
        for i in docs:
            temp = token_stems(i)
            tokens.append(temp)

        x, y = np.asarray(tokens) , np.asarray(train['Type'])
        xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)
        max_len=200
        max_words = 20000
        tok = Tokenizer(num_words=max_words)
        tok.fit_on_texts(x)
        sequences = tok.texts_to_sequences(x)
        test_sequences = tok.texts_to_sequences(xtest)

        
        score = leave_model(name,tok)
        print(score)
        if((score[0][0] > score [0][1])&(score[0][0] > score [0][2])&(score[0][0] > score [0][3])):
            sub_intent = 'LeaveRequest'
        elif((score[0][1] > score [0][0])&(score[0][1] > score [0][2])&(score[0][1] > score [0][3])):
            sub_intent = 'LeaveApproval'
        elif((score[0][2] > score [0][1])&(score[0][2] > score [0][0])&(score[0][2] > score [0][3])):
            sub_intent = 'EmotionlLeave'
        else:
            sub_intent = 'LeaveInquiry'
        return sub_intent



def leave_model(name,tok):
        top_intent = ''
        user_response = name
        max_len=200
        sen = token_stems(user_response)
        sen_test = ([list(sen)])
        print(sen_test)
        sen_sequences = tok.texts_to_sequences(sen_test)
        sen_sequences_matrix = sequence.pad_sequences(sen_sequences,maxlen=max_len)
        m = load_model('Leave_Model_final.h5')
        score = m.predict(sen_sequences_matrix)
        K.clear_session()
            #return("reach till here")
        print(user_response)
        user_response=""
        sen=""
        sen_test=""
        intent=""
        Score=""
        print(score)
        return(score)

In [ ]:
 def classify_leaverequest(name):
        query = name
        sub_type = ''
        train = pd.read_excel("D:\\virtual\\leave_final.xlsx")
        train= train[train['Type']== 'LeaveRequest']
        text = train['Data']
        
        half_day = train[train['sub_type_two'] == 'half_day']
        leave = train[train['sub_type_two'] == 'leave']
        
       
        gen_half_day = [[w.lower() for w in word_tokenize(text)] 
            for text in half_day['Data']]
        gen_leave = [[w.lower() for w in word_tokenize(text)] 
                    for text in leave['Data']]
       


        
        dictionary_half_day = gensim.corpora.Dictionary(gen_half_day)
        dictionary_leave = gensim.corpora.Dictionary(gen_leave)
       
        
        corpus_half_day = [dictionary_half_day.doc2bow(gen_half_day) for gen_half_day in gen_half_day]
        #print(corpus_leave)
        tf_idf_half_day  = gensim.models.TfidfModel(corpus_half_day)
        #print(tf_idf_leave)
        sims_half_day = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\half_day.txt',tf_idf_half_day[corpus_half_day],
                                              num_features=len(dictionary_half_day))



        
        corpus_leave = [dictionary_leave.doc2bow(gen_leave) for gen_leave in gen_leave]
        #print(corpus_leave)
        tf_idf_leave  = gensim.models.TfidfModel(corpus_leave)
        #print(tf_idf_leave)
        sims_leave = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\leave.txt',tf_idf_leave[corpus_leave],
                                              num_features=len(dictionary_leave))
        


       
        query = name
        query_doc = [w.lower() for w in word_tokenize(query)]
        #for approval
        query_doc_bow = dictionary_half_day.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_half_day[query_doc_bow]
        half_day = np.max(sims_half_day[query_doc_tf_idf])
        print(half_day)

        #for reject
        query_doc_bow = dictionary_leave.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_leave[query_doc_bow]
        leave = np.max(sims_leave[query_doc_tf_idf])
        print(leave)
        

       
    
    
        if(half_day>leave):
            return "half_day"
        else:
            return "leave"



# For emotional leave

In [1]:
data = Dataset[Dataset['sub_intent_one'] == 'emotion_LeaveRequest']

NameError: name 'Dataset' is not defined

# Leave Inquiry Model

In [ ]:
train= pd.read_excel("D:\\virtual\\leave_final.xlsx")
train = train[train['Type'] == 'leave_inquiry']
#train = train[train['Module'] == 'Leave']
#train.shape

In [ ]:
docs= train['Data']

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")


def stemming(text):
    
    stems =[stemmer.stem(t) for t in text]
    return stems

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
def tokenizing(text):
    
    #breaking each word and making them tokens
    tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #storing only alpha tokens
    filtered_tokens=[]
    for token in tokens:
        if (re.search('[a-zA-Z]|\'', token)):
            #if token not in stopwords:
            filtered_tokens.append(token)

    return filtered_tokens

In [ ]:
def token_stems(text):  
    tokens=tokenizing(text) 
    stems=stemming(tokens)
    return stems       

In [ ]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)


In [ ]:
x, y = np.asarray(tokens) , np.asarray(train['sub_type_two'])

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
y= to_categorical(y)
#y[:10]

In [ ]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [ ]:
sequences = tok.texts_to_sequences(x)

#test_sequences = tok.texts_to_sequences(xtest)
y = train['sub_type_two']

In [ ]:
xtrain, xtest, ytrain, ytest= train_test_split(sequences, y, test_size= 0.3, random_state=100)

In [ ]:
from sklearn import svm

In [ ]:
model = svm.SVC(C=1,kernel='rbf')

In [ ]:
model.fit(sequences,y)

In [ ]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
#test_sequences_matrix[1]

In [ ]:
def model():
    
    #Create model by simply calling the Sequential constructor
    
    model = Sequential()
    
    #This layer can only be used as the first layer in a model.

    model.add(Embedding(max_words,50,input_length=max_len))
    
    #specify number of neurons in lstm layer and the activation function that we want to use.
    
    #model.add(LSTM(128, input_shape=(xtrain.shape[1:]), activation='tanh', return_sequences=True))
    
    #model.add(Dropout(0.2))
    
    #model.add(LSTM(80, input_shape=(xtrain.shape[1:]), activation='tanh', return_sequences=True))
    
    #model.add(Dropout(0.2))
    
    
    model.add(LSTM(64, activation='tanh',dropout=0.01))
    
    #specify number of neurons in dense layer and the activation function that we want to use.
    
    model.add(Dense(50, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.01))
    
    #and we create our output layer with two nodes as we have 2 class labels.
    
    model.add(Dense(25, activation='relu'))
    
    model.add(Dropout(0.01))
    
    model.add(Dense(5, activation='softmax'))
    
    
    #Now for training, we need to define an optimizer, loss measure and the error metric.
    # We will use the binary_crossentropy as our loss measure. 
    #As for the minimization algorithm, we will use "rmsprop".
    #This optimizer is usually a good choice for recurrent neural networks.
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
c= model()

c.fit(sequences_matrix,y,epochs=50, validation_split=0.1)

In [ ]:
#save your model
c.save('Leave_Inquiry_Model.h5')

# Query

In [ ]:
train= pd.read_excel("D:\\virtual\\leave_final.xlsx")
train = train[train['Type'] == 'leave_inquiry']

In [ ]:
from keras.models import load_model
m = load_model('Leave_Inquiry_Model.h5')
max_len = 300

In [ ]:
docs= train['Data']

In [ ]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)

In [ ]:
x, y = np.asarray(tokens) , np.asarray(train['sub_type_two'])

In [ ]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [ ]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [ ]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [ ]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
query = "update me about leave approving authority"
#query = 'is today a holiday'
#query = 'how many leaves have i taken'
#query = "can you please tell me about my leave encashment"
#query = "is my leave approved"
#query = "will office open tomorrow"
#query = "will office remain close tomorrow"

In [ ]:
#GENRAL LEAVES 1#
query = "is office open today"
query = "is this saturday on"
query = "is tomorrow a working day"
query = "Can you please inform me if today is working day"

In [ ]:
#LEAVE APPROVING AUTHORITY 2#
query = 'who is my leave approving authroity'
query = 'kindly update me about my leave approveing authority'
query = "update me about leave approving authority"
query = 'Can you please update me about my leave approving authority'
query = "inform me who will approve my leaves"

In [ ]:
#SPECIFIC 3#
query = 'how many leaves have i taken'
query = 'how many leaves can i take more'
query = 'how many sick leaves are left of mine'
query = 'can you please tell me how many sick leaves can i take more'
query = 'update me about how many sick leaves can i take more'
query = 'how many leaves has ms.shiza has taken'

In [ ]:
#LEAVE ENCASHMENT 0#
query = "can you please tell me about leave encashment procedure"
query = "how much will i recieve when i will encash my leaves"
query = "what is the procedure of leave encashment"
query = "can you please tell me the procedure of leave encashment"
query = "update me with the procedure of leave encashment"
query = "inform me about the procedure of leave encashment"

In [ ]:
#LEAVE STATUS 4#
query = "is my leave approved"
query = "kindly update me about if my leave is approved or not"
query = "has my sick leave approved"
query = "inform me about if my sick leave is approved"

In [ ]:
#SPECIAL CASES
query = 'is my leave approved'




In [ ]:
sent = query

In [ ]:
sen = token_stems(sent)
sen

In [ ]:
sen_test = ([list(sen)])
sen_sequences = tok.texts_to_sequences(sen_test)
sen_sequences_matrix = sequence.pad_sequences(sen_sequences,maxlen=max_len)

In [ ]:
score = m.predict(sen_sequences_matrix)
print(score)
print(max(score[0]))

In [ ]:
print("leave approve authority 2 2")
print('genral 1 0')
print('specific 3 3')
print('encashment 0 4 0')
print('approval 4 4')

# Code to run

In [ ]:
 def leave_inquiry_module(name):
        sub_intent = ''
        train= pd.read_excel("leave_final.xlsx")
        train = train[train['Type'] == 'leave_inquiry']
        docs= train['Data']
        tokens = []
        for i in docs:
            temp = token_stems(i)
            tokens.append(temp)

        x, y = np.asarray(tokens) , np.asarray(train['sub_type_two'])
        xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)
        max_len=200
        max_words = 20000
        tok = Tokenizer(num_words=max_words)
        tok.fit_on_texts(x)
        sequences = tok.texts_to_sequences(x)
        test_sequences = tok.texts_to_sequences(xtest)

        
        score = leave_inquiry_model(name,tok)
        print(score)
        if((score[0][0] > score [0][1])&(score[0][0] > score [0][2])&(score[0][0] > score [0][3])&(score[0][0] > score [0][4])):
            sub_intent = 'LeaveEncashment'
        elif((score[0][1] > score [0][0])&(score[0][1] > score [0][2])&(score[0][1] > score [0][3])&(score[0][1] > score [0][4])):
            sub_intent = 'LeaveGenral'
        elif((score[0][2] > score [0][1])&(score[0][2] > score [0][0])&(score[0][2] > score [0][3])&(score[0][2] > score [0][4])):
            sub_intent = 'LeaveReporting'
        elif((score[0][3] > score [0][1])&(score[0][3] > score [0][0])&(score[0][3] > score [0][2])&(score[0][3] > score [0][4])):
            sub_intent = 'Leave_Specific'+',"' + leave_inquiry_specific(name) + '"'
        else:
            sub_intent = 'LeaveApproval'
        return sub_intent

In [ ]:
  def leave_inquiry_model(name,tok):
        top_intent = ''
        user_response = name
        max_len=200
        sen = token_stems(user_response)
        sen_test = ([list(sen)])
        print(sen_test)
        sen_sequences = tok.texts_to_sequences(sen_test)
        sen_sequences_matrix = sequence.pad_sequences(sen_sequences,maxlen=max_len)
        m = load_model('Leave_Inquiry_Model.h5')
        score = m.predict(sen_sequences_matrix)
        K.clear_session()
        print(user_response)
        user_response=""
        sen=""
        sen_test=""
        intent=""
        Score=""
        print(score)
        return(score)



# Leave Inquiry Gensim

In [ ]:
 def classify_leaveinquiry(name):
        query = name
        sub_type = ''
        train= pd.read_excel("D:\\virtual\\leave_final.xlsx")
        train= train[train['Type']== 'leave_inquiry']
        text = train['Data']
        
        status = train[train['sub_type_two'] == 'status']
        specific = train[train['sub_type_two'] == 'specific']
        encashment = train[train['sub_type_two'] == 'encashment']
        reporting = train[train['sub_type_two'] == 'reporting']
        genral = train[train['sub_type_two'] == 'genral']
       
        gen_status = [[w.lower() for w in word_tokenize(text)] 
            for text in status['Data']]
        gen_specific = [[w.lower() for w in word_tokenize(text)] 
                    for text in specific['Data']]
        gen_encashment = [[w.lower() for w in word_tokenize(text)] 
                    for text in encashment['Data']]
        gen_reporting = [[w.lower() for w in word_tokenize(text)] 
                    for text in reporting['Data']]
        gen_genral = [[w.lower() for w in word_tokenize(text)] 
                    for text in genral['Data']]



        
        dictionary_status = gensim.corpora.Dictionary(gen_status)
        dictionary_specific = gensim.corpora.Dictionary(gen_specific)
        dictionary_encashment = gensim.corpora.Dictionary(gen_encashment)
        dictionary_reporting = gensim.corpora.Dictionary(gen_reporting)
        dictionary_genral = gensim.corpora.Dictionary(gen_genral)
        
        corpus_status = [dictionary_status.doc2bow(gen_status) for gen_status in gen_status]
        #print(corpus_leave)
        tf_idf_status  = gensim.models.TfidfModel(corpus_status)
        #print(tf_idf_leave)
        sims_status = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\status.txt',tf_idf_status[corpus_status],
                                              num_features=len(dictionary_status))



        
        corpus_specific = [dictionary_specific.doc2bow(gen_specific) for gen_specific in gen_specific]
        #print(corpus_leave)
        tf_idf_specific  = gensim.models.TfidfModel(corpus_specific)
        #print(tf_idf_leave)
        sims_specific = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\specific.txt',tf_idf_specific[corpus_specific],
                                              num_features=len(dictionary_specific))

        
        
        corpus_encashment = [dictionary_encashment.doc2bow(gen_encashment) for gen_encashment in gen_encashment]
        #print(corpus_leave)
        tf_idf_encashment  = gensim.models.TfidfModel(corpus_encashment)
        #print(tf_idf_leave)
        sims_encashment = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\encashment.txt',tf_idf_encashment[corpus_encashment],
                                              num_features=len(dictionary_encashment))
        
        
        corpus_reporting = [dictionary_reporting.doc2bow(gen_reporting) for gen_reporting in gen_reporting]
        #print(corpus_leave)
        tf_idf_reporting  = gensim.models.TfidfModel(corpus_reporting)
        #print(tf_idf_leave)
        sims_reporting = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\reporting.txt',tf_idf_reporting[corpus_reporting],
                                              num_features=len(dictionary_reporting))
        
        
        corpus_genral = [dictionary_genral.doc2bow(gen_genral) for gen_genral in gen_genral]
        #print(corpus_leave)
        tf_idf_genral  = gensim.models.TfidfModel(corpus_genral)
        #print(tf_idf_leave)
        sims_genral = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\genral.txt',tf_idf_genral[corpus_genral],
                                              num_features=len(dictionary_genral))


        


       
        query = name
        query_doc = [w.lower() for w in word_tokenize(query)]
        #for approval
        query_doc_bow = dictionary_status.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_status[query_doc_bow]
        status = np.max(sims_status[query_doc_tf_idf])
        print(status)

        #for reject
        query_doc_bow = dictionary_specific.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_specific[query_doc_bow]
        specific = np.max(sims_specific[query_doc_tf_idf])
        print(specific)
        
        query_doc_bow = dictionary_encashment.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_encashment[query_doc_bow]
        encashment = np.max(sims_encashment[query_doc_tf_idf])
        print(encashment)
        
        query_doc_bow = dictionary_reporting.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_reporting[query_doc_bow]
        reporting = np.max(sims_reporting[query_doc_tf_idf])
        print(reporting)
        
        query_doc_bow = dictionary_genral.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_genral[query_doc_bow]
        genral = np.max(sims_genral[query_doc_tf_idf])
        print(genral)

       
    
    
        if((status>specific)&(status>encashment)&(status>reporting)&(status>genral)):
            return "Leave_status"
        elif((specific>status)&(specific>encashment)&(specific>reporting)&(specific>genral)):
            subintent = leave_inquiry_specific(name)
            return subintent
            #here specific work will come
        elif((encashment>status)&(specific<encashment)&(encashment>reporting)&(encashment>genral)):
            return "Leave_encashment"
        elif((reporting>status)&(reporting>specific)&(encashment<reporting)&(reporting>genral)):
            return "Leave_reporting"
        else:
            return "Genral_holiday"



In [ ]:
name = 'can you please inform me about how many leaves can i take'
name = 'do i have sick leaves'
name = 'how many sick leaves have left'

In [ ]:
print(leave_inquiry_specific(name))

In [ ]:
def leave_inquiry_specific(name):
        query = name
        sub_type = ''
        train = pd.read_excel("D:\\virtual\\leave_final.xlsx")
        train= train[train['sub_type_two']== 'specific']
        text = train['Data']
        
        leave_update = train[train['TypeLeave'] == 'leave_update']
        leave_taken = train[train['TypeLeave'] == 'leave_taken']
        
       
        gen_leave_update = [[w.lower() for w in word_tokenize(text)] 
            for text in leave_update['Data']]
        gen_leave_taken = [[w.lower() for w in word_tokenize(text)] 
                    for text in leave_taken['Data']]

        
        dictionary_leave_update = gensim.corpora.Dictionary(gen_leave_update)
        dictionary_leave_taken = gensim.corpora.Dictionary(gen_leave_taken)
       

        
        corpus_leave_update = [dictionary_leave_update.doc2bow(gen_leave_update) for gen_leave_update in gen_leave_update]
        #print(corpus_leave)
        tf_idf_leave_update  = gensim.models.TfidfModel(corpus_leave_update)
        #print(tf_idf_leave)
        sims_leave_update = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\leave_update.txt',tf_idf_leave_update[corpus_leave_update],
                                              num_features=len(dictionary_leave_update))



        
        corpus_leave_taken = [dictionary_leave_taken.doc2bow(gen_leave_taken) for gen_leave_taken in gen_leave_taken]
        #print(corpus_leave)
        tf_idf_leave_taken  = gensim.models.TfidfModel(corpus_leave_taken)
        #print(tf_idf_leave)
        sims_leave_taken = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\leave_taken.txt',tf_idf_leave_taken[corpus_leave_taken],
                                              num_features=len(dictionary_leave_taken))

        
        

        


       
        query = name
        query_doc = [w.lower() for w in word_tokenize(query)]
        #for approval
        query_doc_bow = dictionary_leave_update.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_leave_update[query_doc_bow]
        leave_update = np.max(sims_leave_update[query_doc_tf_idf])
        print(leave_update)

        #for reject
        query_doc_bow = dictionary_leave_taken.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_leave_taken[query_doc_bow]
        leave_taken = np.max(sims_leave_taken[query_doc_tf_idf])
        print(leave_taken)
        

       
        if(leave_update > leave_taken):
            sub_type = "leave_update"
        else:
            sub_type = "leave_taken"
  
        return(sub_type)
        

            


In [ ]:
def emotional_leave(name):
        query = name
        train= pd.read_excel("D:\\virtual\\leave_final.xlsx")
        sick = train[train["sub_type_two"] == 'sick']
        casual = train[train["sub_type_two"] == 'casual']


        #raw_documents = train['Leave Data Description']
        #print("Number of sick documents:",len(sick))
        #print("Number of casual documents:",len(casual))
        gen_docs_s = [[w.lower() for w in word_tokenize(text)] 
                for text in sick['Data']]
        gen_docs_c = [[w.lower() for w in word_tokenize(text)] 
                for text in casual['Data']]
        dictionary_s = gensim.corpora.Dictionary(gen_docs_s)
        dictionary_c = gensim.corpora.Dictionary(gen_docs_c)
        corpus_s = [dictionary_s.doc2bow(gen_doc_s) for gen_doc_s in gen_docs_s]
        #print(corpus_l)

        corpus_c = [dictionary_c.doc2bow(gen_doc_c) for gen_doc_c in gen_docs_c]
        #print(corpus_i)
        
        tf_idf_s = gensim.models.TfidfModel(corpus_s)
        tf_idf_c = gensim.models.TfidfModel(corpus_c)
        sims_s = gensim.similarities.Similarity('D:\\gensim\\sick.txt',tf_idf_s[corpus_s],
                                          num_features=len(dictionary_s))
        query_doc = [w.lower() for w in word_tokenize(query)]
        #print(query_doc)
        query_doc_bow = dictionary_s.doc2bow(query_doc)
        #print(query_doc_bow)
        query_doc_tf_idf = tf_idf_s[query_doc_bow]
        sick = np.max(sims_s[query_doc_tf_idf])
        print("sick:",np.max(sims_s[query_doc_tf_idf]))
        sims_c = gensim.similarities.Similarity('D:\gensim\\casual.txt',tf_idf_c[corpus_c],
                                          num_features=len(dictionary_c))
        query_doc_bow = dictionary_c.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_c[query_doc_bow]
        casual = np.max(sims_c[query_doc_tf_idf])
        print("Casual:",np.max(sims_c[query_doc_tf_idf]))
        if(casual>sick):
            return('CasualLeave')
        else:
            return('Indirect_SickLeave')






In [ ]:
def leave_approval(name):
        query = name
        sub_type = ''
        train = pd.read_excel("leave_final.xlsx")
        train = train[train['Type'] == 'Leave_Approval']
        text = train['Data']
        approve = train[train['sub_type_two'] == 'approve']
        reject = train[train['sub_type_two'] == 'reject']
       
        gen_approve = [[w.lower() for w in word_tokenize(text)] 
            for text in approve['Data']]
        gen_reject = [[w.lower() for w in word_tokenize(text)] 
                    for text in reject['Data']]

        
        dictionary_approve = gensim.corpora.Dictionary(gen_approve)
        dictionary_reject = gensim.corpora.Dictionary(gen_reject)


        
        corpus_approve = [dictionary_approve.doc2bow(gen_approve) for gen_approve in gen_approve]
        #print(corpus_leave)
        tf_idf_approve  = gensim.models.TfidfModel(corpus_approve)
        #print(tf_idf_leave)
        sims_approve = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\approve.txt',tf_idf_approve[corpus_approve],
                                              num_features=len(dictionary_approve))



        
        corpus_reject = [dictionary_reject.doc2bow(gen_reject) for gen_reject in gen_reject]
        #print(corpus_leave)
        tf_idf_reject  = gensim.models.TfidfModel(corpus_reject)
        #print(tf_idf_leave)
        sims_reject = gensim.similarities.Similarity('C:\\Users\\shiza.abid\\Desktop\\reject.txt',tf_idf_reject[corpus_reject],
                                              num_features=len(dictionary_reject))


        


       
        query = name
        query_doc = [w.lower() for w in word_tokenize(query)]
        #for approval
        query_doc_bow = dictionary_approve.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_approve[query_doc_bow]
        approve = np.max(sims_approve[query_doc_tf_idf])
        print(approve)

        #for reject
        query_doc_bow = dictionary_reject.doc2bow(query_doc)
        query_doc_tf_idf = tf_idf_reject[query_doc_bow]
        reject = np.max(sims_reject[query_doc_tf_idf])
        print(reject)

       
        if(approve > reject):
            sub_type = sub_type + "approve"
        else:
            sub_type = sub_type + "reject"

        return(sub_type)



    


# LEAVE ENCASHMENT 

In [ ]:

sub_type = ''
train = pd.read_excel("D:\\virtual\\leave_final.xlsx")
leave_encashment = train[train['TypeLeave'] == 'leave_encashment']
leave_encashment_policy = train[train['TypeLeave'] == 'leave_encashment_policy']
leave_encashment_procedure = train[train['TypeLeave'] == 'leave_encashment_procedure']
    
gen_docs_leave_encashment = [[w.lower() for w in word_tokenize(text)] 
            for text in leave_encashment['Data']]
gen_docs_leave_encashment_policy = [[w.lower() for w in word_tokenize(text)] 
            for text in leave_encashment_policy['Data']]
gen_docs_leave_encashment_procedure = [[w.lower() for w in word_tokenize(text)] 
            for text in leave_encashment_procedure['Data']]
    
dictionary_leave_encashment  = gensim.corpora.Dictionary(gen_docs_leave_encashment )
dictionary_leave_encashment_policy = gensim.corpora.Dictionary(gen_docs_leave_encashment_policy)
dictionary_leave_encashment_procedure = gensim.corpora.Dictionary(gen_docs_leave_encashment_procedure)
    
corpus_leave_encashment = [dictionary_leave_encashment.doc2bow(gen_doc_leave_encashment) for gen_doc_leave_encashment in gen_docs_leave_encashment]
corpus_leave_encashment_policy = [dictionary_leave_encashment_policy.doc2bow(gen_doc_leave_encashment_policy) for gen_doc_leave_encashment_policy in gen_docs_leave_encashment_policy]
corpus_leave_encashment_procedure = [dictionary_leave_encashment_procedure.doc2bow(gen_doc_leave_encashment_procedure) for gen_doc_leave_encashment_procedure in gen_docs_leave_encashment_procedure]

tf_idf_leave_encashment = gensim.models.TfidfModel(corpus_leave_encashment)
tf_idf_leave_encashment_policy = gensim.models.TfidfModel(corpus_leave_encashment_policy)
tf_idf_leave_encashment_procedure = gensim.models.TfidfModel(corpus_leave_encashment_procedure )
    
sims_leave_encashment = gensim.similarities.Similarity('D:\\bot_Django\\virtual\\virtual_bot\\bot\\Gensim_data\\leave_encashment.txt',tf_idf_leave_encashment[corpus_leave_encashment],
                                num_features=len(dictionary_leave_encashment))
sims_leave_encashment_policy = gensim.similarities.Similarity('D:\\bot_Django\\virtual\\virtual_bot\\bot\\Gensim_data\\leave_encashment_policy.txt',tf_idf_leave_encashment_policy[corpus_leave_encashment_policy],
                                num_features=len(dictionary_leave_encashment_policy))
sims_leave_encashment_procedure = gensim.similarities.Similarity('D:\\bot_Django\\virtual\\virtual_bot\\bot\\Gensim_data\\leave_encashment_procedure.txt',tf_idf_leave_encashment_procedure[corpus_leave_encashment_procedure],
                                num_features=len(dictionary_leave_encashment_procedure))
    
leave_encashment_gensim={1:sims_leave_encashment,2:tf_idf_leave_encashment,3:dictionary_leave_encashment,4:sims_leave_encashment_policy,5:tf_idf_leave_encashment_policy,6:dictionary_leave_encashment_policy,7:sims_leave_encashment_procedure,8:tf_idf_leave_encashment_procedure,9:dictionary_leave_encashment_procedure}

name = 'how can i encash my leaves'
name = 'inform me how can i encash my leaves'
name = 'what is leave encashment'
name = 'tell me about leave encashment'
leave_encashment_classification(name,leave_encashment_gensim)


In [ ]:
def leave_encashment_classification(name,leave_encashment_gensim):
    query = name
    sims_leave_encashment = leave_encashment_gensim[1]
    tf_idf_leave_encashment  = leave_encashment_gensim[2]
    dictionary_leave_encashment = leave_encashment_gensim[3]
    sims_leave_encashment_policy = leave_encashment_gensim[4]
    tf_idf_leave_encashment_policy = leave_encashment_gensim[5]
    dictionary_leave_encashment_policy = leave_encashment_gensim[6]
    sims_leave_encashment_procedure = leave_encashment_gensim[7]
    tf_idf_leave_encashment_procedure = leave_encashment_gensim[8]
    dictionary_leave_encashment_procedure = leave_encashment_gensim[9]
    
    query_doc = [w.lower() for w in word_tokenize(query)]
    
    query_doc_bow = dictionary_leave_encashment.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf_leave_encashment[query_doc_bow]
    leave_encashment = np.max(sims_leave_encashment[query_doc_tf_idf])
    
    query_doc_bow = dictionary_leave_encashment_policy.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf_leave_encashment_policy[query_doc_bow]
    leave_encashment_policy = np.max(sims_leave_encashment_policy[query_doc_tf_idf])
    
    query_doc_bow = dictionary_leave_encashment_procedure.doc2bow(query_doc)
    query_doc_tf_idf = tf_idf_leave_encashment_procedure[query_doc_bow]
    leave_encashment_procedure = np.max(sims_leave_encashment_procedure[query_doc_tf_idf])
    
    if((leave_encashment > leave_encashment_policy)&(leave_encashment > leave_encashment_procedure)):
        print('leave_encashment')
    elif((leave_encashment_policy>leave_encashment)&(leave_encashment_policy>leave_encashment_procedure)):
        print('leave_encashment_policy')
    else:
        print('leave_encashment_procedure')
        
    
    



# Leave Encashment Model

In [ ]:
train = pd.read_excel('D:\\bot_Django\\virtual\\virtual_bot\\bot\\datasets\\leave_final.xlsx')

In [ ]:
train = train[train['sub_type_two'] == 'encashment']

In [ ]:
docs= train['Data']

In [ ]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)


In [ ]:
x, y = np.asarray(tokens) , np.asarray(train['TypeLeave'])

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
y= to_categorical(y)

In [ ]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=50)

In [ ]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [ ]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [ ]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
def model():
    
    #Create model by simply calling the Sequential constructor
    
    model = Sequential()
    
    #This layer can only be used as the first layer in a model.
    
    model.add(Embedding(max_words,50,input_length=max_len))
    
    #specify number of neurons in lstm layer and the activation function that we want to use.
    
    model.add(LSTM(64, activation='tanh',dropout=0.01))
    
    #specify number of neurons in dense layer and the activation function that we want to use.
    
    model.add(Dense(50, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.01))
    
    #and we create our output layer with two nodes as we have 2 class labels.
    model.add(Dense(25, activation='relu'))
    
    #model.add(Dense(25, activation='relu'))
    
    
    model.add(Dropout(0.01))
    
    
    model.add(Dense(3, activation='softmax'))
    
    #Now for training, we need to define an optimizer, loss measure and the error metric.
    # We will use the binary_crossentropy as our loss measure. 
    #As for the minimization algorithm, we will use "rmsprop".
    #This optimizer is usually a good choice for recurrent neural networks.
    
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
    return model

In [ ]:
c= model()

c.fit(sequences_matrix,y,epochs=50, validation_split=0.1)

In [ ]:
c.save('Encashment_Model.h5')

In [ ]:
train = pd.read_excel('D:\\bot_Django\\virtual\\virtual_bot\\bot\\datasets\\leave_final.xlsx')
train = train[train['sub_type_two'] == 'encashment']

In [ ]:
from keras.models import load_model
m = load_model('Encashment_Model.h5')
max_len = 300

In [ ]:
docs= train['Data']

In [ ]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)

In [ ]:
x, y = np.asarray(tokens) , np.asarray(train['TypeLeave'])

In [ ]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [ ]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [ ]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [ ]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
query = 'How many Leave allowance will I get?'
query = 'Do i have any leaves that can be encashed'
query = 'How much i will get if i cash my leaves'
query = 'can i encash my sick leaves '
query = 'can my sick leaves be encashed '
query = 'can i encash my hajj leaves'

In [ ]:
query = 'what is leave encashment policy'
query = 'tell me about my leave encashment '
query = 'what is leave encashment policy'

In [ ]:
query = 'can you tell me the procedure of leave encashment'
query = 'tell me about encashment of leaves '

In [ ]:
sent = query

In [ ]:
sen = token_stems(sent)
sen

In [ ]:
sen_test = ([list(sen)])
sen_sequences = tok.texts_to_sequences(sen_test)
sen_sequences_matrix = sequence.pad_sequences(sen_sequences,maxlen=max_len)

In [ ]:
score = m.predict(sen_sequences_matrix)
print(score)
print(max(score[0]))

In [ ]:
print('leave allowance 0')
print('policy 1')
print('procedure 2')